BM25

In [1]:
!pip install -U beir
!pip install 'elasticsearch<7.14.0'

  Using cached beir-0.2.2-py3-none-any.whl (49 kB)
  Using cached sentence-transformers-2.1.0.tar.gz (78 kB)
  Using cached pytrec_eval-0.5.tar.gz (15 kB)
  Using cached tensorflow_text-2.6.0-cp37-cp37m-manylinux1_x86_64.whl (4.4 MB)
  Using cached faiss_cpu-1.7.1.post2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.4 MB)
  Using cached elasticsearch-7.15.1-py2.py3-none-any.whl (378 kB)
  Using cached transformers-4.11.3-py3-none-any.whl (2.9 MB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached sentencepiece-0.1.96-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached huggingface_hub-0.0.19-py3-none-any.whl (56 kB)
  Using cached sacremoses-0.0.46-py3-none-any.whl (895 kB)
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
  Created wheel for pytrec-eval: filena

In [2]:
import pathlib, os
import time
import pandas as pd
from beir import util
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
if not os.path.exists('elasticsearch-oss-7.9.2-linux-x86_64.tar.gz'):
  !wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
  !wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
  !tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
  !sudo chown -R daemon:daemon elasticsearch-7.9.2/
  !shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 

--2021-10-16 11:04:40--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229941304 (219M) [application/x-gzip]
Saving to: ‘elasticsearch-oss-7.9.2-linux-x86_64.tar.gz’

elasticsearch-oss-7 100%[===================>] 219.29M  12.4MB/s    in 22s     

2021-10-16 11:05:02 (9.95 MB/s) - ‘elasticsearch-oss-7.9.2-linux-x86_64.tar.gz’ saved [229941304/229941304]

--2021-10-16 11:05:02--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 20

In [4]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

Starting job # 0 in a separate thread.


In [5]:
time.sleep(20)

In [6]:
!ps -ef | grep elasticsearch

root         688     686  0 11:05 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       689     688 93 11:05 ?        00:00:18 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-4026014966230851529 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecou

In [7]:
!curl -sX GET "localhost:9200/"

{
  "name" : "230dbf18aedf",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "EP_fkn7pS_eP0GOYc4jcIw",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [8]:
def retrieve(dataset):
  hostname = 'localhost' 
  url = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip'
  out_dir = os.path.join(os.getcwd(), 'datasets')
  dataset_r = dataset
  index_r = dataset_r
  data_path_r = util.download_and_unzip(url.format(dataset_r), out_dir)
  corpus_r, queries_r, qrels_r = GenericDataLoader(data_path_r).load(split='test') # 'test', 'train', 'dev'
  model_r = BM25(index_name=index_r, hostname=hostname, initialize=True) # initialize=True : reindex
  retriever_r = EvaluateRetrieval(model_r)
  results_r = retriever_r.retrieve(corpus_r, queries_r)
  return retriever_r.evaluate(qrels_r, results_r, retriever_r.k_values)

In [9]:
def evaluate(dataset_name, model_name):
  ndcg, _map, recall, precision = retrieve(dataset_name)
  c_map = 'MAP@10'
  c_map_h = 'MAP@100'
  c_map_t = 'MAP@1000'
  c_ndcg = 'NDCG@10'
  c_ndcg_h = 'NDCG@100'
  c_ndcg_t = 'NDCG@1000'
  c_pre = 'P@10'
  c_pre_h = 'P@100'
  c_pre_t = 'P@1000'
  c_rec = 'Recall@10'
  c_rec_h = 'Recall@100'
  c_rec_t = 'Recall@1000'
  eval_dict = {
      c_map: [_map[c_map]], 
      c_map_h: [_map[c_map_h]],
      c_map_t: [_map[c_map_t]],
      c_ndcg: [ndcg[c_ndcg]], 
      c_ndcg_h: [ndcg[c_ndcg_h]],
      c_ndcg_t: [ndcg[c_ndcg_t]],
      c_pre: [precision[c_pre]], 
      c_pre_h: [precision[c_pre_h]],
      c_pre_t: [precision[c_pre_t]],
      c_rec: [recall[c_rec]],
      c_rec_h: [recall[c_rec_h]],
      c_rec_t: [recall[c_rec_t]]}
  eval_df = pd.DataFrame(data=eval_dict)
  eval_df.index = [model_name]
  return eval_df

In [10]:
evaluate('webis-touche2020', 'TOUCHE')

/content/datasets/webis-touche2020.zip:   0%|          | 0.00/217M [00:00<?, ?iB/s]

  0%|          | 0/382545 [00:00<?, ?it/s]

que: 100%|██████████| 1/1 [00:11<00:00, 11.01s/it]


,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
TOUCHE,0.13872,0.2211,0.23848,0.34707,0.45483,0.55796,0.33061,0.09531,0.01635,0.2122,0.56093,0.882


In [11]:
evaluate('arguana', 'ARGUANA')

/content/datasets/arguana.zip:   0%|          | 0.00/3.60M [00:00<?, ?iB/s]

  0%|          | 0/8674 [00:00<?, ?it/s]

que: 100%|██████████| 11/11 [01:26<00:00,  7.83s/it]


,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
ARGUANA,0.38394,0.39441,0.39462,0.47163,0.5167,0.52186,0.07532,0.00952,0.00099,0.7532,0.95164,0.99147


In [12]:
evaluate('fiqa', 'FIQA')

/content/datasets/fiqa.zip:   0%|          | 0.00/17.1M [00:00<?, ?iB/s]

  0%|          | 0/57638 [00:00<?, ?it/s]

que: 100%|██████████| 6/6 [00:35<00:00,  5.97s/it]


,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
FIQA,0.19102,0.20421,0.2059,0.25365,0.31255,0.34704,0.07022,0.01304,0.00192,0.3244,0.54731,0.75728


end of fun.